<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/340_WDO_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Workforce Development Orchestrator — Goal & Planning Nodes

This section defines how the Workforce Development Orchestrator **decides what it is trying to accomplish** and **how it will execute that work**. Rather than relying on implicit model reasoning, these nodes establish a transparent, rule-driven control layer that governs the entire agent.

At a high level:

* The **goal node** defines *why* the agent is running
* The **planning node** defines *how* the agent will proceed

Both are intentionally deterministic and auditable.

---

## 1. Goal Definition as a Control Mechanism

The `goal_node` translates user input into a structured, machine-readable objective.

Instead of allowing free-form interpretation, the agent operates within one of two clearly defined scopes:

* **Organization-wide analysis**
* **Single-employee analysis**

This design choice ensures the agent’s intent is:

* Explicit
* Inspectable
* Stable across executions

### Why This Matters

Goals are not hidden inside prompts or inferred implicitly. They are stored in state and carried through the entire workflow, which allows:

* Downstream nodes to adapt behavior based on scope
* Reports to explain *what* was analyzed and *why*
* Auditors to verify that the system did what it claimed to do

In practice, this prevents the common failure mode where an agent “drifts” from its original purpose during execution.

---

## 2. Rule-Based Planning Instead of Model Guessing

The `planning_node` converts the goal into a **step-by-step execution plan**. Each step explicitly declares:

* Its purpose
* Its dependencies
* The outputs it is responsible for producing

This is a deliberate rejection of opaque, model-generated plans in favor of **deterministic orchestration**.

### Key Design Strengths

#### 📐 Fixed, Predictable Execution

Every run follows the same logical structure:

1. Load data
2. Analyze risk
3. Detect skill gaps
4. Recommend learning paths
5. Assess role evolution
6. Prioritize outcomes
7. Generate a report

This predictability is essential for:

* Debugging
* Testing
* Executive trust

---

#### 🔗 Explicit Dependencies

Each step declares what it depends on. This ensures:

* Correct sequencing
* No hidden coupling between nodes
* Clear failure boundaries

If a dependency fails, the system can stop or degrade gracefully without producing misleading results.

---

#### 📦 Declared Outputs

Each step lists the state fields it produces. This turns the plan into a **data contract**, making it obvious:

* Where each piece of intelligence comes from
* Which node owns which responsibility

This dramatically improves maintainability and accountability.

---

## 3. Thin Nodes, Strong Utilities

The nodes themselves do not implement business logic. They:

* Define intent
* Route execution
* Update state

All real computation is delegated to utilities. This separation:

* Keeps orchestration readable
* Makes logic reusable across agents
* Allows utilities to be tested independently

This is a hallmark of scalable agent systems.

---

## 4. Error Handling as a First-Class Concern

Both nodes preserve and extend the shared `errors` list rather than overwriting it. This allows:

* Multiple failures to be captured in a single run
* Partial execution without silent data corruption
* Clear post-run diagnostics

This is especially important for multi-step, enterprise-facing agents.

---

## Why This Design Inspires Confidence

This goal-and-plan structure ensures the orchestrator:

* Always knows what it is doing
* Always knows why it is doing it
* Never hides decision flow inside model behavior

For leaders, this means:

* Clear scope
* Predictable outcomes
* Auditable execution

For developers, it means:

* Clean separation of concerns
* Easy extensibility
* Low cognitive load when debugging or enhancing the system




# Orchestration nodes for Workforce Development Orchestrator Agent

In [ ]:
"""Orchestration nodes for Workforce Development Orchestrator Agent

Following the pattern: Nodes orchestrate, utilities implement.
Each node is thin and delegates work to utilities.
"""

from typing import Dict, Any
from config import WorkforceDevelopmentOrchestratorState, WorkforceDevelopmentOrchestratorConfig


def goal_node(state: WorkforceDevelopmentOrchestratorState) -> Dict[str, Any]:
    """
    Goal Node: Define the goal for workforce development analysis.

    This is a simple rule-based goal definition that sets the framework.
    """
    employee_id = state.get("employee_id")

    if employee_id is None:
        # Analyze all employees
        goal = {
            "objective": "Analyze workforce development needs across all employees",
            "scope": "all_employees",
            "focus_areas": [
                "automation_risk_analysis",
                "skill_gap_detection",
                "learning_path_recommendations",
                "role_evolution_planning"
            ]
        }
    else:
        # Analyze specific employee
        goal = {
            "objective": f"Analyze workforce development needs for employee {employee_id}",
            "scope": "single_employee",
            "employee_id": employee_id,
            "focus_areas": [
                "automation_risk_analysis",
                "skill_gap_detection",
                "learning_path_recommendations",
                "role_evolution_planning"
            ]
        }

    return {
        "goal": goal,
        "errors": state.get("errors", [])
    }


def planning_node(state: WorkforceDevelopmentOrchestratorState) -> Dict[str, Any]:
    """
    Planning Node: Create execution plan based on goal.

    This creates a step-by-step plan. Rule-based, no LLM needed.
    """
    goal = state.get("goal")

    if not goal:
        return {
            "errors": state.get("errors", []) + ["planning_node: goal is required"]
        }

    plan = [
        {
            "step": 1,
            "name": "data_loading",
            "description": "Load employee, role, task, skill, and related data from JSON files",
            "dependencies": [],
            "outputs": [
                "employees",
                "roles",
                "tasks",
                "skills",
                "automation_signals",
                "skill_gaps",
                "learning_paths",
                "role_evolution",
                "roles_lookup",
                "tasks_lookup",
                "skills_lookup",
                "employees_lookup",
                "tasks_by_role",
                "employees_by_role"
            ]
        },
        {
            "step": 2,
            "name": "automation_risk_analysis",
            "description": "Analyze automation risk for tasks and roles",
            "dependencies": ["data_loading"],
            "outputs": ["automation_risk_analysis"]
        },
        {
            "step": 3,
            "name": "skill_gap_detection",
            "description": "Detect and analyze skill gaps for employees",
            "dependencies": ["data_loading"],
            "outputs": ["skill_gap_analysis"]
        },
        {
            "step": 4,
            "name": "learning_path_matching",
            "description": "Match employees to appropriate learning paths",
            "dependencies": ["skill_gap_detection"],
            "outputs": ["learning_path_recommendations"]
        },
        {
            "step": 5,
            "name": "role_evolution_analysis",
            "description": "Analyze role evolution opportunities",
            "dependencies": ["data_loading", "automation_risk_analysis"],
            "outputs": ["role_evolution_recommendations"]
        },
        {
            "step": 6,
            "name": "prioritization",
            "description": "Prioritize gaps, learning paths, and role evolutions",
            "dependencies": [
                "skill_gap_detection",
                "learning_path_matching",
                "role_evolution_analysis"
            ],
            "outputs": [
                "prioritized_gaps",
                "prioritized_recommendations",
                "prioritized_evolutions",
                "workforce_summary"
            ]
        },
        {
            "step": 7,
            "name": "report_generation",
            "description": "Generate comprehensive workforce development report",
            "dependencies": ["prioritization"],
            "outputs": ["workforce_report", "report_file_path"]
        }
    ]

    return {
        "plan": plan,
        "errors": state.get("errors", [])
    }



# Test foundation nodes for Workforce Development Orchestrator

In [ ]:
"""Test foundation nodes for Workforce Development Orchestrator

Testing Phase 1: Goal and Planning nodes
Following the pattern: Test each component before proceeding
"""

from agents.workforce_development_orchestrator.nodes import goal_node, planning_node
from config import WorkforceDevelopmentOrchestratorState


def test_goal_node_all_employees():
    """Test goal node with all employees scope"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": None,
        "errors": []
    }

    result = goal_node(state)

    assert "goal" in result
    assert result["goal"]["scope"] == "all_employees"
    assert result["goal"]["objective"] == "Analyze workforce development needs across all employees"
    assert "automation_risk_analysis" in result["goal"]["focus_areas"]
    assert len(result.get("errors", [])) == 0

    print("✅ test_goal_node_all_employees: PASSED")


def test_goal_node_single_employee():
    """Test goal node with single employee scope"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": "E001",
        "errors": []
    }

    result = goal_node(state)

    assert "goal" in result
    assert result["goal"]["scope"] == "single_employee"
    assert result["goal"]["employee_id"] == "E001"
    assert len(result.get("errors", [])) == 0

    print("✅ test_goal_node_single_employee: PASSED")


def test_planning_node():
    """Test planning node creates proper execution plan"""
    state: WorkforceDevelopmentOrchestratorState = {
        "goal": {
            "objective": "Analyze workforce development needs",
            "scope": "all_employees"
        },
        "errors": []
    }

    result = planning_node(state)

    assert "plan" in result
    assert len(result["plan"]) == 7
    assert result["plan"][0]["name"] == "data_loading"
    assert result["plan"][-1]["name"] == "report_generation"
    assert len(result.get("errors", [])) == 0

    print("✅ test_planning_node: PASSED")


def test_planning_node_requires_goal():
    """Test planning node requires goal"""
    state: WorkforceDevelopmentOrchestratorState = {
        "errors": []
    }

    result = planning_node(state)

    assert "errors" in result
    assert len(result["errors"]) > 0
    assert "planning_node: goal is required" in result["errors"]

    print("✅ test_planning_node_requires_goal: PASSED")


def test_goal_and_planning_together():
    """Test goal and planning nodes work together"""
    state: WorkforceDevelopmentOrchestratorState = {
        "employee_id": None,
        "errors": []
    }

    # First goal node
    state = goal_node(state)
    assert "goal" in state

    # Then planning node
    state = planning_node(state)
    assert "plan" in state
    assert len(state["plan"]) == 7

    print("✅ test_goal_and_planning_together: PASSED")


if __name__ == "__main__":
    print("=" * 60)
    print("Testing Foundation Nodes (Phase 1)")
    print("=" * 60)
    print()

    test_goal_node_all_employees()
    test_goal_node_single_employee()
    test_planning_node()
    test_planning_node_requires_goal()
    test_goal_and_planning_together()

    print()
    print("=" * 60)
    print("✅ All foundation tests passed!")
    print("=" * 60)



In [ ]:
(.venv) micahshull@Micahs-iMac AI_AGENTS_008_Workforce_Development_Orchestrator % python3 test_workforce_development_foundation.py
============================================================
Testing Foundation Nodes (Phase 1)
============================================================

✅ test_goal_node_all_employees: PASSED
✅ test_goal_node_single_employee: PASSED
✅ test_planning_node: PASSED
✅ test_planning_node_requires_goal: PASSED
✅ test_goal_and_planning_together: PASSED

============================================================
✅ All foundation tests passed!
============================================================
